In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Change working directory to be current folder
import os
os.chdir('/content/drive/MyDrive/ExerciseScoring')
!ls

Mounted at /content/drive
 best_model_ex2.h5		   person2_ex1_NPr_P.mp4
'best model ex4'		   person2_ex1_Pr_NP.mp4
 best_model_ex4.h5		   person2_ex1_Pr_P.mp4
 Data				   predict.ipynb
 demo.ipynb			  'pretrain model'
 demo_predict2.ipynb		   README.md
 demo_predict.ipynb		   rehabilitation_ex2_improved.json
 demo.py			   rehabilitation_ex2.json
 Figure				   rehabilitation_ex4.json
 GCN				   requirements.txt
 Kimore_ex2			   test_model_ex2.h5
 Kimore_ex4			   test_predict.ipynb
'KIMORE Preprocessed'		   train.ipynb
 LICENSE			   train.py
'Model architecture.drawio'	   withSpec_nopain_correctpose.mp4
 noSpec_nopain_correctpose.mp4	   withSpec_nopain_correctpose_score.pkl
 noSpec_nopain_incorrectpose.mp4   withSpec_nopain_incorrectpose.mp4
 noSpec_pain_correctpose.mp4	   withSpec_pain_correctpose.mp4
 noSpec_pain_incorrectpose.mp4	   withSpec_pain_incorrectpose.mp4
 person2_ex1_NPr_NP.mp4		   withSpec_pain_incorrectpose_score.pkl


In [ ]:
!pip install tensorflow_addons

import numpy as np
random_seed = 42  # for reproducibility
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import concatenate, Flatten, Dropout, Dense, Input, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
from sklearn.model_selection import train_test_split
# from IPython.core.debugger import set_trace
#import matplotlib.pyplot as plt
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from GCN.data_processing import Data_Loader
from GCN.graph import Graph
from GCN.sgcn_lstm import Sgcn_Lstm
from sklearn.metrics import mean_squared_error, mean_absolute_error

import argparse
import joblib

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


# Actual training

In [ ]:
"""import the whole dataset"""
data_loader = Data_Loader('Kimore_ex2')  # folder name -> Train.csv, Test.csv

"""import the graph data structure"""
graph = Graph(len(data_loader.body_part))

"""Split the data into training and validation sets while preserving the distribution"""
train_x, test_x, train_y, test_y = train_test_split(data_loader.scaled_x, data_loader.scaled_y, test_size=0.2, random_state = random_seed)

"""Train the algorithm"""
algorithm = Sgcn_Lstm(train_x, train_y, graph.AD, graph.AD2, graph.bias_mat_1, graph.bias_mat_2, lr = 0.0001, epoach=150, batch_size=10)
history = algorithm.train()

"""Test the model"""
y_pred = algorithm.prediction(test_x)
y_pred = data_loader.sc2.inverse_transform(y_pred)
test_y = data_loader.sc2.inverse_transform(test_y)

"""Save u scaling object"""
joblib.dump(data_loader.sc2, 'Data/Ex2_sc_y.save', compress=True)
print('Scaling objects saved!')

Epoch 1/150


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


30/30 [==============================] - ETA: 0s - loss: 0.0711
Epoch 1: val_loss improved from inf to 0.06154, saving model to /content/drive/MyDrive/STGCN-rehab-main/best_model_ex2.h5
30/30 [==============================] - 147s 5s/step - loss: 0.0711 - val_loss: 0.0615
Epoch 2/150
30/30 [==============================] - ETA: 0s - loss: 0.0629
Epoch 2: val_loss improved from 0.06154 to 0.05896, saving model to /content/drive/MyDrive/STGCN-rehab-main/best_model_ex2.h5
30/30 [==============================] - 97s 3s/step - loss: 0.0629 - val_loss: 0.0590
Epoch 3/150
30/30 [==============================] - ETA: 0s - loss: 0.0604
Epoch 3: val_loss improved from 0.05896 to 0.05647, saving model to /content/drive/MyDrive/STGCN-rehab-main/best_model_ex2.h5
30/30 [==============================] - 98s 3s/step - loss: 0.0604 - val_loss: 0.0565
Epoch 4/150
30/30 [==============================] - ETA: 0s - loss: 0.0582
Epoch 4: val_loss improved from 0.05647 to 0.05546, saving model to /con

In [ ]:
"""Save the model architecture"""
algorithm = Sgcn_Lstm(train_x, train_y, graph.AD, graph.AD2, graph.bias_mat_1, graph.bias_mat_2, lr = 0.0001, epoach=150, batch_size=10)
model_file = algorithm.model_save()

#save the model architecture to JSON file
with open('rehabilitation_ex2.json', 'w') as json_file:
    json_file.write(model_file)

In [ ]:
"""Performance matric"""
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
  
test_dev = abs(test_y-y_pred)
mean_abs_dev = np.mean(test_dev)
mae = mean_absolute_error(test_y, y_pred)
rms_dev = sqrt(mean_squared_error(y_pred, test_y))
mse = mean_squared_error(test_y,y_pred) 
mape = mean_absolute_percentage_error(test_y, y_pred)
print('Mean absolute deviation:', mae)
print('RMS deviation:', rms_dev)
print('MSE:', mse)
print('MAPE: ', mape)

Mean absolute deviation: 3.3463596257124113
RMS deviation: 5.52903576580146
MSE: 30.570236499511733
MAPE:  11.315425236619006
